In [5]:
import overpass, osmapi

In [90]:
# open condos file
f = open("condos.csv","r").readlines()

# start not found condo file
j = open("condos-left.csv","w")

# set username
username = 'schleuss'

# crack open APIs
api = overpass.API()
api2 = osmapi.OsmApi(passwordfile = "/Users/jschleuss/Documents/osm.password")

In [97]:
# find id by ain
def findBuildings(ain):
    ain = ain.replace('\n','')
    print 'way["lacounty:ain"="'+ain+'"]'
    
    # example condo parcel
    response = api.Get('(way["lacounty:ain"="'+str(ain)+'"];relation["lacounty:ain"="'+str(ain)+'"];)')
    
    print str(len(response['features'])) + ' total features found for AIN ' + ain
    print response
    

In [92]:
def updateBuildings(ain):

    # WILL NEED TO LOOP THROUGH POSSIBLE MULTIPLE BUILDINGS
    ain = ain.replace('\n','')
    response = api.Get('way["lacounty:ain"="'+str(ain)+'"]')

    # alt if way gives nothing
    if len(response['features']) == 0:
        response = api.Get('way["lacounty:ain"="'+ain+'"]')
        objid = response['features'][0]['id']

    else:
        objid = response['features'][0]['id']
        
    obj = api2.WayGet(objid) # get object

    # open a changeset
    api2.ChangesetCreate({u"comment": u"Removing building:units from condo and adding apartments type", u'source':'L.A. County Assessor', u'created_by':username})
    
    # delete unit count and change building to residential 
    if 'building:units' in obj['tag']:
        del obj['tag']['building:units']
        print 'updated units for ' + str(objid)
    if obj['tag']['building'] != 'apartments':
        obj['tag']['building'] = 'apartments'
        print 'updated building type for ' + str(objid)
        
    obj = api2.WayUpdate(obj)

In [98]:
# testing to find buildings
selain = f[2]
findBuildings(selain)

way["lacounty:ain"="7563006036"]
8 total features found for AIN 7563006036
{"features": [{"geometry": {"coordinates": [[-118.3222191, 33.7259609], [-118.3222148, 33.7259601], [-118.3222417, 33.7258504], [-118.3224172, 33.7258797], [-118.3223894, 33.7259929], [-118.3223795, 33.7259912], [-118.3223625, 33.7260605], [-118.3223445, 33.7261269], [-118.3221854, 33.7260969], [-118.3222023, 33.7260299], [-118.3222191, 33.7259609]], "type": "LineString"}, "id": 407337799, "properties": {"building": "apartments", "ele": "121.2", "height": "5.2", "lacounty:ain": "7563006036", "lacounty:bld_id": "463670722850", "start_date": "1984"}, "type": "Feature"}, {"geometry": {"coordinates": [[-118.3221726, 33.7255784], [-118.3223519, 33.7255673], [-118.3223543, 33.7255947], [-118.322527, 33.725584], [-118.3225245, 33.7255558], [-118.3227022, 33.7255448], [-118.3227178, 33.7257219], [-118.3225434, 33.7257327], [-118.322546, 33.7257616], [-118.3223645, 33.7257728], [-118.322362, 33.7257443], [-118.3221882, 3

In [ ]:
# testing to update buildings
updateBuildings(selain)